In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # add linear operation
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 2x2 max pooling
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 方阵可用 2 代替 （2,2）
        x = F.max_pool2d(F.relu((self.conv2(x))), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [19]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [20]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [21]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0570,  0.0231,  0.0537,  0.0269,  0.0612, -0.0861, -0.0852, -0.0520,
          0.0092,  0.0258]], grad_fn=<AddmmBackward>)


In [22]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [23]:
output = net(input)
target = torch.randn(10)
print(target)
target = target.view(1, -1)
print(target)
crierion = nn.MSELoss()

loss = crierion(output, target)
print(loss)

tensor([-1.5362, -1.6183,  1.9510,  0.2369,  1.4579,  0.6859,  0.9607,  1.5067,
        -1.1792, -0.3759])
tensor([[-1.5362, -1.6183,  1.9510,  0.2369,  1.4579,  0.6859,  0.9607,  1.5067,
         -1.1792, -0.3759]])
tensor(1.6520, grad_fn=<MseLossBackward>)


In [11]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [24]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward(retain_graph=True)

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0019,  0.0007, -0.0035,  0.0164,  0.0089,  0.0060])
